In [7]:
import latentvelo as ltv
import numpy as np
import scanpy as sc
import scvelo as scv
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import anndata as ad

SEED = 2024
np.random.seed(SEED)

In [8]:
adata = sc.read_h5ad("/media/liyaru/LYR/Benchmark/Redeem/redeem_aged.h5ad")
print(adata)
scv.pp.filter_and_normalize(adata,n_top_genes=2000)
scv.pp.moments(adata)
print(adata)
adata.write_h5ad("/media/liyaru/LYR/Diff_change/11_redeem_aged/adata/aged2.h5ad")

AnnData object with n_obs × n_vars = 8907 × 36601
    obs: 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_SCT', 'nFeature_SCT', 'SCT.weight', 'ATAC.weight', 'seurat_clusters', 'Sig.HSC1', 'Sig.Prog1', 'Sig.EarlyE1', 'Sig.LateE1', 'Sig.ProMono1', 'Sig.Mono1', 'Sig.ncMono1', 'Sig.cDC1', 'Sig.pDC1', 'Sig.ProB1', 'Sig.PreB1', 'Sig.B1', 'Sig.Plasma1', 'Sig.T1', 'Sig.CTL1', 'Sig.NK1', 'STD.CellType', 'STD_Cat', 'STD_Cat2', 'Sample', 'MitoCoverage', 'ClonalGroup', 'ClonalGroup.Prob', 'wsnn_res.0.8', 'Origin.Seurat', 'Celltype', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'nCount_ambiguous', 'nFeature_ambiguous'
    var: 'name'
    obsm: 'X_pca', 'X_umap'
    layers: 'ambiguous', 'spliced', 'unspliced'
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.
computing neighbors


/home/liyaru/miniconda3/envs/LatentVelo/lib/python3.8/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


    finished (0:00:01) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
AnnData object with n_obs × n_vars = 8907 × 2000
    obs: 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_SCT', 'nFeature_SCT', 'SCT.weight', 'ATAC.weight', 'seurat_clusters', 'Sig.HSC1', 'Sig.Prog1', 'Sig.EarlyE1', 'Sig.LateE1', 'Sig.ProMono1', 'Sig.Mono1', 'Sig.ncMono1', 'Sig.cDC1', 'Sig.pDC1', 'Sig.ProB1', 'Sig.PreB1', 'Sig.B1', 'Sig.Plasma1', 'Sig.T1', 'Sig.CTL1', 'Sig.NK1', 'STD.CellType', 'STD_Cat', 'STD_Cat2', 'Sample', 'MitoCoverage', 'ClonalGroup', 'ClonalGroup.Prob', 'wsnn_res.0.8', 'Origin.Seurat', 'Celltype', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'nCount_ambiguous', 'nFeature_ambiguous', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_coun

In [3]:
# adata = ltv.utils.standard_clean_recipe(adata,normalize_library = False)
adata = ltv.utils.standard_clean_recipe(adata)
adata.var['velocity_genes'] = True

spliced_key = 'spliced'
unspliced_key = 'unspliced'

spliced_library_sizes = adata.layers[spliced_key].sum(1)
unspliced_library_sizes = adata.layers[unspliced_key].sum(1)

if len(spliced_library_sizes.shape) == 1:
       spliced_library_sizes = spliced_library_sizes[:,None]
if len(unspliced_library_sizes.shape) == 1:
       unspliced_library_sizes = unspliced_library_sizes[:,None]

adata.obs['spliced_size_factor'] = spliced_library_sizes #spliced_all_size_factors
adata.obs['unspliced_size_factor'] = unspliced_library_sizes #unspliced_all_size_factors

model = ltv.models.VAE(observed = 2000) # observed: number of genes
epochs, val_ae, val_traj = ltv.train(model, adata,name="LatentVelo")


Extracted 2000 highly variable genes.
Choosing top 2000 genes


/home/liyaru/miniconda3/envs/LatentVelo/lib/python3.8/site-packages/latentvelo-0.1-py3.8.egg/latentvelo/utils.py:153: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  scv.pp.log1p(adata)


computing neighbors
    finished (0:00:02) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:01) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
Warning, folder already exists. This may overwrite a previous fit.
2000 velocity genes used
epoch 0, full loss 125.719, val loss 33.770, recon MSE 0.180, traj MSE 0.018, reg loss -1.645
epoch 1, full loss 75.551, val loss 11.265, recon MSE 0.152, traj MSE 0.013, reg loss -2.122
epoch 2, full loss 78.807, val loss -7.877, recon MSE 0.102, traj MSE 0.013, reg loss -2.035
epoch 3, full loss 106.346, val loss -27.023, recon MSE 0.054, traj MSE 0.012, reg loss -2.335
epoch 4, full loss 157.960, val loss -45.428, recon MSE 0.030, traj MSE 0.012, reg loss -2.669
epoch 5, full loss 240.784, val loss -61.679, recon MSE 0.022, traj MSE 0.012, reg loss -2.703
epoch 6, full loss 307.889, val loss -74.931, recon MSE 0.018, traj

In [4]:
latent_adata, adata = ltv.output_results(model, adata, gene_velocity=True,
                                         embedding='umap')


In [5]:
print(adata)
print(latent_adata)

AnnData object with n_obs × n_vars = 8907 × 2000
    obs: 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_SCT', 'nFeature_SCT', 'SCT.weight', 'ATAC.weight', 'seurat_clusters', 'Sig.HSC1', 'Sig.Prog1', 'Sig.EarlyE1', 'Sig.LateE1', 'Sig.ProMono1', 'Sig.Mono1', 'Sig.ncMono1', 'Sig.cDC1', 'Sig.pDC1', 'Sig.ProB1', 'Sig.PreB1', 'Sig.B1', 'Sig.Plasma1', 'Sig.T1', 'Sig.CTL1', 'Sig.NK1', 'STD.CellType', 'STD_Cat', 'STD_Cat2', 'Sample', 'MitoCoverage', 'ClonalGroup', 'ClonalGroup.Prob', 'wsnn_res.0.8', 'Origin.Seurat', 'Celltype', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'nCount_ambiguous', 'nFeature_ambiguous', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'spliced_size_factor', 'unspliced_size_factor', 'batch_id', 'celltype_id', 'root', 'terminal', 'latent_time'
    var: 'name', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_genes', 'R2', 'R2_traj', 'R2_test

In [6]:
adata.write_h5ad("adata/latentvelo.h5ad")
latent_adata.write_h5ad("adata/latentvelo_latent.h5ad")